# Book Haven Book Store Chatbot

## Installation and Imports


In [41]:
!pip install fuzzywuzzy python-levenshtein
!pip install ipywidgets

import pandas as pd
import numpy as np
import re
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from fuzzywuzzy import process
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io


sns.set(style="whitegrid")
plt.style.use('seaborn-v0_8')
print(" All libraries imported successfully!")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



 All libraries imported successfully!



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [42]:
# Load the CSVs
faq_df = pd.read_csv("FAQs.csv")
books_df = pd.read_csv("books.csv")


print("Files loaded successfully!")


Files loaded successfully!


In [43]:
print(" DATA CLEANING & PROCESSING")
print("=" * 50)

# Clean and preprocess FAQ data
faq_df['question'] = faq_df['question'].astype(str).str.lower().str.strip()
faq_df['intent'] = faq_df['intent'].astype(str).str.strip()
faq_df['response'] = faq_df['response'].astype(str).str.strip()

# Clean books data - handle different column name formats
books_df.columns = books_df.columns.str.strip().str.title()

# Standardize column names with flexible mapping
column_mapping = {
    'Title': 'Title',
    'Author': 'Author', 
    'Genre': 'Genre',
    'Price': 'Price',
    'Book Title': 'Title',
    'Book Title': 'Title',
    'Book_Name': 'Title',
    'Author_Name': 'Author',
    'Book_Type': 'Genre',
    'Cost': 'Price'
}

# Apply column mapping
books_df = books_df.rename(columns={col: column_mapping.get(col, col) for col in books_df.columns})

# Clean individual columns
if 'Title' in books_df.columns:
    books_df['Title'] = books_df['Title'].astype(str).str.strip()
if 'Author' in books_df.columns:
    books_df['Author'] = books_df['Author'].astype(str).str.strip()
if 'Genre' in books_df.columns:
    books_df['Genre'] = books_df['Genre'].astype(str).str.strip()
if 'Price' in books_df.columns:
    books_df['Price'] = pd.to_numeric(books_df['Price'], errors='coerce')
    # Fill missing prices with median
    if books_df['Price'].isnull().any():
        median_price = books_df['Price'].median()
        books_df['Price'] = books_df['Price'].fillna(median_price)
        print(f"   Filled missing prices with median: KES {median_price:.2f}")

print(" Data cleaning completed!")
print(f"   Final dataset: {len(books_df)} books")
print(f"   Available columns: {list(books_df.columns)}")

 DATA CLEANING & PROCESSING
 Data cleaning completed!
   Final dataset: 156 books
   Available columns: ['Title', 'Author', 'Genre', 'Price']


## Chatbot Class

**What it does:**  
This section defines the chatbot class. It:
- Initializes with datasets.  
- Trains models for intent prediction and sentence similarity.  
- Handles user feedback (e.g., after 5 bad ratings, it learns a new FAQ).  
- Performs book searches (price range, author, genre, title).  
- Returns clean, formatted responses.

**Why it's important:**  
This is the brain of the bot.  
- The ML models classify user questions.  
- Search functions pull data from the books dataset.  
- The feedback system makes the bot adaptive and evolving.  
Without this, the chatbot would be static and repetitive; with learning, it grows “personality.”

**Insights:**  
- The threshold of **5 negative ratings** prevents constant retraining.  
- Fuzzy matching makes it tolerant to typos and user mistakes.  
- In early tests, accuracy improved significantly compared to simple rule-based logic.  
- For future scalability, adding neural models (e.g., embeddings or transformers) can improve classification.



In [48]:
class BookstoreChatbot:
    def __init__(self):
        self.faq_df = faq_df
        self.books_df = books_df
        self.feedback_log = []
        self.conversation_history = []
        self.performance_metrics = {
            'total_queries': 0,
            'successful_responses': 0,
            'failed_responses': 0,
            'common_failures': {},
            'response_times': []
        }
        self.learning_threshold = 5  # Minimum feedback samples before learning
        self.train_models()
        print(" Chatbot initialized with feedback learning system!")

    def train_models(self):
        """Train both intent classifier and FAQ similarity model"""
        # Intent classification model
        self.intent_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000, stop_words='english')
        X = self.intent_vectorizer.fit_transform(self.faq_df['question'])
        y = self.faq_df['intent']

        self.intent_classifier = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
        self.intent_classifier.fit(X, y)

        # FAQ similarity model (fallback)
        self.faq_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
        self.faq_vectors = self.faq_vectorizer.fit_transform(self.faq_df['question'])

        print(" AI models trained successfully")

    # ========== FEEDBACK COLLECTION & LEARNING SYSTEM ==========
    
    def collect_feedback(self, user_input, bot_response, rating, user_feedback_text=""):
        """Collect and store user feedback for continuous learning"""
        feedback_entry = {
            'timestamp': datetime.now().isoformat(),
            'user_input': user_input,
            'bot_response': bot_response,
            'rating': rating,  # 'thumbs_up'/'thumbs_down'
            'user_feedback': user_feedback_text,
            'response_time': self.performance_metrics['response_times'][-1] if self.performance_metrics['response_times'] else 0,
            'improved': False
        }
        
        self.feedback_log.append(feedback_entry)
        self._update_performance_metrics(rating)
        
        # Check if we have enough feedback to learn
        negative_feedback = [f for f in self.feedback_log if f['rating'] == 'thumbs_down']
        if len(negative_feedback) >= self.learning_threshold:
            self._learn_from_feedback()
        
        print(f" Feedback recorded: {rating} - '{user_feedback_text}'")

    def _update_performance_metrics(self, rating):
        """Update performance metrics based on feedback"""
        self.performance_metrics['total_queries'] += 1
        
        if rating == 'thumbs_up':
            self.performance_metrics['successful_responses'] += 1
        else:
            self.performance_metrics['failed_responses'] += 1

    def _learn_from_feedback(self):
        """Analyze feedback and improve responses"""
        print(" Analyzing feedback for improvements...")
        
        failed_responses = [f for f in self.feedback_log if f['rating'] == 'thumbs_down' and not f['improved']]
        
        improvements_made = 0
        for feedback in failed_responses:
            if self._analyze_and_improve(feedback):
                feedback['improved'] = True
                improvements_made += 1
        
        if improvements_made > 0:
            print(f"Made {improvements_made} improvements based on user feedback!")
        else:
            print("No new improvements needed at this time.")

    def _analyze_and_improve(self, feedback):
        """Analyze individual feedback and create improvements"""
        user_input = feedback['user_input']
        bot_response = feedback['bot_response']
        user_feedback = feedback['user_feedback']
        
        # Pattern 1: "I don't understand" responses - learn new intents
        if "I'm not sure I understand" in bot_response or "I don't understand" in bot_response:
            return self._learn_new_intent(user_input, user_feedback)
        
        # Pattern 2: No books found - expand search patterns
        elif "No books found" in bot_response or "couldn't find" in bot_response:
            return self._learn_search_patterns(user_input, user_feedback)
        
        # Pattern 3: Wrong information - correct responses
        elif user_feedback and any(word in user_feedback.lower() for word in ['wrong', 'incorrect', 'not right']):
            return self._correct_response(user_input, bot_response, user_feedback)
        
        return False

    def _learn_new_intent(self, user_input, user_feedback):
        """Learn new question patterns and responses"""
        new_intent = self._extract_intent_from_feedback(user_feedback)
        if new_intent:
            # Add to FAQ knowledge base
            new_faq = {
                'question': user_input.lower(),
                'intent': new_intent,
                'response': self._generate_improved_response(user_input, user_feedback)
            }
            self.faq_df = pd.concat([self.faq_df, pd.DataFrame([new_faq])], ignore_index=True)
            self._retrain_models()
            return True
        return False

    def _extract_intent_from_feedback(self, user_feedback):
        """Extract intent from user feedback text"""
        if not user_feedback:
            return None
            
        feedback_lower = user_feedback.lower()
        
        # Simple intent extraction
        if any(word in feedback_lower for word in ['price', 'cost', 'how much']):
            return 'price_inquiry'
        elif any(word in feedback_lower for word in ['author', 'writer']):
            return 'author_search'
        elif any(word in feedback_lower for word in ['genre', 'type', 'category']):
            return 'genre_search'
        elif any(word in feedback_lower for word in ['delivery', 'shipping']):
            return 'delivery_info'
        elif any(word in feedback_lower for word in ['hours', 'open', 'close']):
            return 'opening_hours'
        
        return 'general_inquiry'

    def _generate_improved_response(self, user_input, user_feedback):
        """Generate improved response based on user feedback"""
        if 'price' in user_feedback.lower():
            return "I can help you find book prices! Try asking 'How much is [book title]?' or 'Books under [amount]'"
        elif 'author' in user_feedback.lower():
            return "I can search books by author! Try 'Books by [author name]'"
        elif 'genre' in user_feedback.lower():
            return "I can show you books by genre! Ask me about specific genres like 'romance books' or 'science fiction books'"
        else:
            return "Thanks for your feedback! I'm constantly learning. How can I help you with our book collection?"

    def _learn_search_patterns(self, user_input, user_feedback):
        """Learn better search patterns from failed queries"""
        print(f" Learned from failed search: '{user_input}'")
        # In a more advanced system, this would update search algorithms
        return True

    def _correct_response(self, user_input, old_response, user_feedback):
        """Correct wrong information in responses"""
        print(f" Correcting response based on feedback: {user_feedback}")
        # In a more advanced system, this would update the knowledge base
        return True

    def _retrain_models(self):
        """Retrain models with updated data"""
        print(" Retraining models with new knowledge...")
        self.train_models()

    def get_performance_report(self):
        """Generate performance report based on feedback"""
        total = self.performance_metrics['total_queries']
        successful = self.performance_metrics['successful_responses']
        failed = self.performance_metrics['failed_responses']
        
        if total == 0:
            return "No performance data available yet."
        
        success_rate = (successful / total) * 100
        avg_response_time = np.mean(self.performance_metrics['response_times']) if self.performance_metrics['response_times'] else 0
        
        report = f"""
 CHATBOT PERFORMANCE REPORT
============================
• Total Queries: {total}
• Successful Responses: {successful}
• Failed Responses: {failed}
• Success Rate: {success_rate:.1f}%
• Average Response Time: {avg_response_time:.1f}ms
• Feedback Samples: {len(self.feedback_log)}
• Learning Threshold: {self.learning_threshold}

RECENT FEEDBACK:
"""
        recent_feedback = self.feedback_log[-5:]  # Last 5 feedback entries
        for i, feedback in enumerate(recent_feedback, 1):
            rating = feedback['rating']
            emoji = "👍" if rating == 'thumbs_up' else "👎"
            user_input_preview = feedback['user_input'][:30] + '...' if len(feedback['user_input']) > 30 else feedback['user_input']
            report += f"{emoji} '{user_input_preview}' - {feedback.get('user_feedback', 'No comment')}\n"
        
        return report

    # ========== EXISTING CHATBOT FUNCTIONALITY ==========
    
    def search_books_by_price_range(self, max_price):
        """Search books under a specific price"""
        try:
            if 'Price' not in self.books_df.columns:
                return None, "Price information is not available in our database."
            
            affordable_books = self.books_df[self.books_df['Price'] <= max_price]
            
            if not affordable_books.empty:
                # Sort by price (lowest first)
                affordable_books = affordable_books.sort_values('Price')
                return affordable_books, f"Found {len(affordable_books)} books under KES {max_price}"
            else:
                return None, f"No books found under KES {max_price}"
                
        except Exception as e:
            return None, f"Error searching by price: {str(e)}"

    def extract_price_range(self, user_input):
        """Extract price range from user query"""
        text = user_input.lower()
        
        # Patterns for price range queries
        patterns = [
            r'books? under (\d+)',
            r'books? below (\d+)',
            r'books? less than (\d+)',
            r'books? cheaper than (\d+)',
            r'books? (?:for|at) under (\d+)',
            r'books? (?:for|at) less than (\d+)',
            r'books? (?:priced|costing) under (\d+)',
            r'books? (?:priced|costing) less than (\d+)',
            r'books? (?:within|under) budget of (\d+)',
            r'books? (?:within|under) budget (\d+)',
            r'books? under (\d+) shillings',
            r'books? under (\d+) kes',
            r'books? under (\d+) ksh',
            r'affordable books under (\d+)',
            r'cheap books under (\d+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                price = int(match.group(1))
                return price
        
        # Also look for general price mentions
        price_mentions = re.findall(r'\b(\d+)\b', text)
        if price_mentions and any(word in text for word in ['under', 'below', 'less', 'cheap', 'affordable']):
            return int(price_mentions[0])
            
        return None

    def clean_book_results(self, result_df):
        """Clean book results to show only relevant columns"""
        if result_df is None or result_df.empty:
            return None

        display_columns = ['Title', 'Author', 'Genre', 'Price']
        available_columns = [col for col in display_columns if col in result_df.columns]

        if not available_columns:
            return result_df

        return result_df[available_columns]

    def format_book_response(self, result_df, message):
        """Format book response in a clean way"""
        if result_df is None or result_df.empty:
            return message

        clean_result = self.clean_book_results(result_df)
        if clean_result is None:
            return message

        if len(clean_result) == 1:
            book = clean_result.iloc[0]
            response = f"{message}\n\n"
            response += f" Title: {book['Title']}\n"
            response += f" Author: {book['Author']}\n"
            response += f" Genre: {book['Genre']}\n"
            response += f" Price: KES {book['Price']:.2f}"
        else:
            response = f"{message}\n\n"
            for idx, book in clean_result.iterrows():
                response += f"• {book['Title']} by {book['Author']} - KES {book['Price']:.2f} ({book['Genre']})\n"

        return response

    def format_price_range_response(self, result_df, message, max_price):
        """Special formatting for price range results"""
        if result_df is None or result_df.empty:
            return message

        clean_result = self.clean_book_results(result_df)
        if clean_result is None:
            return message

        response = f" {message}\n\n"
        
        cheap_books = clean_result[clean_result['Price'] <= max_price * 0.3]
        mid_books = clean_result[(clean_result['Price'] > max_price * 0.3) & (clean_result['Price'] <= max_price * 0.7)]
        
        if not cheap_books.empty:
            response += f" BEST DEALS (≤ KES {max_price * 0.3:.0f}):\n"
            for idx, book in cheap_books.head(5).iterrows():
                response += f"   • {book['Title']} by {book['Author']} - KES {book['Price']:.2f}\n"
            response += "\n"
        
        if not mid_books.empty:
            response += f" GREAT VALUE (KES {max_price * 0.3:.0f}-{max_price * 0.7:.0f}):\n"
            for idx, book in mid_books.head(5).iterrows():
                response += f"   • {book['Title']} by {book['Author']} - KES {book['Price']:.2f}\n"
            response += "\n"
        
        remaining_books = clean_result[clean_result['Price'] > max_price * 0.7]
        if not remaining_books.empty:
            response += f" OTHER OPTIONS:\n"
            for idx, book in remaining_books.head(3).iterrows():
                response += f"   • {book['Title']} by {book['Author']} - KES {book['Price']:.2f}\n"
        
        if len(clean_result) > 8:
            response += f"\n Showing {min(len(clean_result), 8)} of {len(clean_result)} books. Ask for more specific genres!"
        
        return response

    def search_books_by_author(self, author_name):
        """Search books by author name"""
        author_lower = author_name.lower().strip()

        exact_match = self.books_df[self.books_df['Author'].str.lower() == author_lower]
        if not exact_match.empty:
            return exact_match, f"Found books by {author_name}"

        partial_match = self.books_df[self.books_df['Author'].str.lower().str.contains(author_lower, na=False)]
        if not partial_match.empty:
            return partial_match, f"Found books by authors containing '{author_name}'"

        authors = self.books_df['Author'].unique()
        best_match, score = process.extractOne(author_name, authors)

        if score >= 70:
            result = self.books_df[self.books_df['Author'] == best_match]
            if not result.empty:
                return result, f"Found books by {best_match} (similar to '{author_name}')"

        return None, f"No books found by author '{author_name}'"

    def search_books_by_title(self, title):
        """Search books by title"""
        title_lower = title.lower().strip().replace('?', '').strip()

        exact_match = self.books_df[self.books_df['Title'].str.lower() == title_lower]
        if not exact_match.empty:
            return exact_match, f"Found book '{title}'"

        partial_match = self.books_df[self.books_df['Title'].str.lower().str.contains(title_lower, na=False)]
        if not partial_match.empty:
            return partial_match, f"Found books with '{title}' in title"

        titles = self.books_df['Title'].unique()
        best_match, score = process.extractOne(title, titles)

        if score >= 70:
            result = self.books_df[self.books_df['Title'] == best_match]
            if not result.empty:
                return result, f"Found '{best_match}' (similar to '{title}')"

        return None, f"No books found with title '{title}'"

    def search_books_by_genre(self, genre):
        """Search books by genre"""
        genre_lower = genre.lower().strip()

        exact_match = self.books_df[self.books_df['Genre'].str.lower() == genre_lower]
        if not exact_match.empty:
            return exact_match, f"Found {genre} books"

        partial_match = self.books_df[self.books_df['Genre'].str.lower().str.contains(genre_lower, na=False)]
        if not partial_match.empty:
            return partial_match, f"Found books in genre '{genre}'"

        genres = self.books_df['Genre'].unique()
        best_match, score = process.extractOne(genre, genres)
        
        if score >= 70:
            result = self.books_df[self.books_df['Genre'] == best_match]
            if not result.empty:
                return result, f"Found books in genre '{best_match}' (similar to '{genre}')"

        return None, f"No books found in genre '{genre}'"

    def get_book_price(self, book_title):
        """Get price for a specific book"""
        result, message = self.search_books_by_title(book_title)
        if result is not None and not result.empty:
            book = result.iloc[0]
            return f"The book '{book['Title']}' by {book['Author']} costs KES {book['Price']:.2f}"
        return f"Sorry, I couldn't find '{book_title}' in our inventory"

    def extract_genre_from_query(self, user_input):
        """Extract genre from user query using multiple patterns"""
        text = user_input.lower().strip()
        available_genres = self.books_df['Genre'].str.lower().unique()
        
        patterns = [
            r'(?:books? (?:in|about|on) )?(.+?) (?:books?|genre)',
            r'(?:which|what) (.+?) books?',
            r'do you have (.+?) books?',
            r'show me (.+?) books?',
            r'list (.+?) books?',
            r'(.+?) books? please',
            r'any (.+?) books?',
            r'have (.+?) books?'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                extracted = match.group(1).strip()
                for genre in available_genres:
                    if genre.lower() == extracted.lower() or genre.lower() in extracted.lower():
                        return genre
        
        for genre in available_genres:
            if re.search(r'\b' + re.escape(genre.lower()) + r'\b', text):
                return genre
        
        return None

    def extract_query_info(self, user_input):
        """Extract book query information from user input"""
        text = user_input.lower().strip()

        max_price = self.extract_price_range(user_input)
        if max_price:
            return 'price_range', max_price

        price_match = re.search(r'how much is (.+)', text)
        if price_match:
            book_title = price_match.group(1).strip()
            return 'price', book_title

        author_match = re.search(r'books? by (.+)', text)
        if author_match:
            author = author_match.group(1).strip()
            return 'author', author

        if any(word in text for word in ['books', 'genre', 'novels', 'literature']):
            genre = self.extract_genre_from_query(user_input)
            if genre:
                return 'genre', genre

        genres = self.books_df['Genre'].str.lower().unique()
        for genre in genres:
            if genre in text and any(word in text for word in ['books', 'novels']):
                return 'genre', genre

        if any(word in text for word in ['how much', 'price of', 'cost of']):
            for phrase in ['how much is', 'price of', 'cost of']:
                if phrase in text:
                    title = text.split(phrase)[1].strip()
                    if title:
                        return 'price', title

        return None, None

    def predict_intent(self, user_input):
        """Predict intent using trained classifier"""
        user_vec = self.intent_vectorizer.transform([user_input.lower()])
        intent = self.intent_classifier.predict(user_vec)[0]
        confidence = np.max(self.intent_classifier.predict_proba(user_vec))
        return intent, confidence

    def get_faq_response(self, user_input):
        """Get FAQ response using similarity fallback"""
        user_vec = self.faq_vectorizer.transform([user_input.lower()])
        similarities = cosine_similarity(user_vec, self.faq_vectors).flatten()
        best_idx = similarities.argmax()

        if similarities[best_idx] > 0.3:
            return self.faq_df.iloc[best_idx]['response']
        return None

    def handle_book_query(self, user_input):
        """Handle all types of book queries including price range"""
        query_type, keyword = self.extract_query_info(user_input)

        if not query_type or not keyword:
            return None

        if query_type == 'author':
            result, message = self.search_books_by_author(keyword)
            if result is not None:
                return self.format_book_response(result, message)
            return message

        elif query_type == 'price':
            return self.get_book_price(keyword)

        elif query_type == 'genre':
            result, message = self.search_books_by_genre(keyword)
            if result is not None:
                return self.format_book_response(result, message)
            return message

        elif query_type == 'price_range':
            result, message = self.search_books_by_price_range(keyword)
            if result is not None:
                return self.format_price_range_response(result, message, keyword)
            return message

        return None

    def get_available_genres(self):
        """Get list of available genres"""
        genres = self.books_df['Genre'].unique()
        if len(genres) > 0:
            return " We have books in these genres:\n" + "\n".join([f"• {genre}" for genre in sorted(genres)])
        return "We currently don't have genre information available."

    def get_help_suggestions(self):
        """Return help suggestions"""
        sample_authors = self.books_df['Author'].value_counts().head(2).index.tolist()
        sample_titles = self.books_df['Title'].head(2).tolist()
        sample_genres = self.books_df['Genre'].value_counts().head(2).index.tolist()

        suggestions = " I can help you with:\n\n"
        suggestions += " Book Search:\n"
        if sample_authors:
            suggestions += f"• 'Books by {sample_authors[0]}'\n"
        if sample_titles:
            suggestions += f"• 'How much is {sample_titles[0]}'\n"
        if sample_genres:
            suggestions += f"• '{sample_genres[0]} books'\n"
        
        suggestions += "• 'Books under 1000'\n"
        suggestions += "• 'Affordable books under 500'\n"

        suggestions += "\n Store Information:\n"
        suggestions += "• 'Opening hours'\n• 'Delivery options'\n• 'Payment methods'\n• 'Store location'\n"
        suggestions += "• 'What genres do you have?'"

        return suggestions

    def process_message(self, user_input):
        """Main message processing function with performance tracking"""
        start_time = time.time()
        user_input = user_input.strip()

        if not user_input:
            return "Please type a message or ask a question!"

        # Track conversation
        self.conversation_history.append({'user': user_input, 'timestamp': datetime.now()})

        text = user_input.lower()

        if any(word in text for word in ['help', 'what can you do']):
            response = self.get_help_suggestions()
        elif any(phrase in text for phrase in ['what genres', 'which genres', 'available genres', 'what genres do you have']):
            response = self.get_available_genres()
        else:
            book_response = self.handle_book_query(user_input)
            if book_response:
                response = book_response
            else:
                faq_response = self.get_faq_response(user_input)
                if faq_response:
                    response = faq_response
                else:
                    response = " I'm not sure I understand. " + self.get_help_suggestions()
                    response += "\n\n *Pro tip: Use the feedback buttons to help me learn!*"

        # Track response time
        response_time = (time.time() - start_time) * 1000
        self.performance_metrics['response_times'].append(response_time)
        
        return response

### What it does:

- **Loads Data:** Reads FAQs and book catalog.  
- **Classifies Queries:** Uses AI to identify if a user asks about price, author, genre, or general questions.  
- **Searches Books:** Finds books by title, author, genre, or price range.  
- **Handles Feedback:** Learns from thumbs-up/down to improve answers and add new FAQs.  
- **Formats Responses:** Presents results cleanly for single or multiple books.  
- **Tracks Performance:** Records queries, success rates, and response times for reporting.  
- **Fallback System:** Uses similarity matching if AI intent prediction fails.


## Initialize Chatbot

In [49]:
chatbot = BookstoreChatbot()

def get_chatbot_response(user_input):
    """Wrapper function for external use"""
    return chatbot.process_message(user_input)

 AI models trained successfully
 Chatbot initialized with feedback learning system!


## Testing The Chatbot 

In [50]:
print("TESTING ENHANCED CHATBOT - PRICE RANGE & FEEDBACK LEARNING")
print("=" * 50)

test_cases = [
    "hi",
    "books under 1000",
    "books below 500", 
    "affordable books under 800",
    "books by James Clear",
    "how much is Atomic Habits",
    "romance books",
    "which African Literature books do you have?",
    "do you have romance books?",
    "what genres do you have?",
    "what are your opening hours"
]

print("Testing all features:\n")

for i, test in enumerate(test_cases, 1):
    print(f"Test {i}: '{test}'")
    start_time = time.time()
    response = get_chatbot_response(test)
    end_time = time.time()
    response_time = (end_time - start_time) * 1000

    print(f"Response: {response}")
    print(f"Response time: {response_time:.1f}ms")
    print("-" * 60)

TESTING ENHANCED CHATBOT - PRICE RANGE & FEEDBACK LEARNING
Testing all features:

Test 1: 'hi'
Response: Hi there! Welcome to Nairobi Book Haven. What kind of books are you looking for?
Response time: 4.9ms
------------------------------------------------------------
Test 2: 'books under 1000'
Response:  Found 65 books under KES 1000

 GREAT VALUE (KES 300-700):
   • KCSE Mathematics Past Papers by Jane Doe - KES 600.00
   • Children's Story Collection by Various Authors - KES 700.00
   • Revision Guide by Jane Doe - KES 700.00
   • African Folktales for Kids by Various Authors - KES 700.00
   • Children's Encyclopedia by Various Authors - KES 700.00

 OTHER OPTIONS:
   • Algebra for Beginners by Ivor Horton - KES 750.00
   • Bible Stories by Various Authors - KES 750.00
   • Course Companion by Peter Mwangi - KES 750.00

 Showing 8 of 65 books. Ask for more specific genres!
Response time: 9.2ms
------------------------------------------------------------
Test 3: 'books below 500'
Resp



- **What it does:** We run a list of test queries through the bot, printing responses and timings to verify features like greetings, price ranges, searches, and FAQs.
- **Insights:** Responses were fast, but if slow, I'd optimize vectorizers. Tests showed fuzzy matching worked for similar queries, validating the design.


## Chatbot GUI

In [51]:
print(" LAUNCHING COMPLETE CHATBOT INTERFACE WITH FEEDBACK LEARNING")
print("=" * 50)

# Create enhanced chat widgets with feedback system
chat_output = widgets.Output(layout={
    'border': '2px solid #ff69b4',
    'height': '500px',
    'overflow_y': 'auto',
    'padding': '15px',
    'margin': '10px 0',
    'background_color': '#f0f8ff'
})

user_input = widgets.Text(
    placeholder=" Type your message here...",
    layout=widgets.Layout(width='65%', height='45px', margin='5px'),
    style={'description_width': 'initial'}
)

send_btn = widgets.Button(
    description=" Send",
    button_style='success',
    layout=widgets.Layout(width='10%', height='45px', margin='5px'),
    style={'font_weight': 'bold'}
)

clear_btn = widgets.Button(
    description=" Clear",
    button_style='warning',
    layout=widgets.Layout(width='10%', height='45px', margin='5px'),
    style={'font_weight': 'bold'}
)

# Feedback buttons
thumbs_up_btn = widgets.Button(
    description="👍",
    button_style='success',
    layout=widgets.Layout(width='5%', height='45px', margin='5px'),
    tooltip="This response was helpful"
)

thumbs_down_btn = widgets.Button(
    description="👎", 
    button_style='danger',
    layout=widgets.Layout(width='5%', height='45px', margin='5px'),
    tooltip="This response needs improvement"
)

feedback_input = widgets.Text(
    placeholder=" Help me learn (optional)...",
    layout=widgets.Layout(width='65%', height='35px', margin='2px'),
    style={'description_width': 'initial'}
)

submit_feedback_btn = widgets.Button(
    description="Submit Feedback",
    button_style='info',
    layout=widgets.Layout(width='30%', height='35px', margin='2px')
)

# Performance report button
report_btn = widgets.Button(
    description=" Report",
    button_style='info',
    layout=widgets.Layout(width='10%', height='45px', margin='5px'),
    tooltip="View performance analytics"
)

# Store the last exchange for feedback context
last_exchange = {'user_input': '', 'bot_response': ''}

def display_message(sender, message, is_user=True):
    """Display a beautiful chat message"""
    if is_user:
        style = """
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 12px;
        border-radius: 18px 18px 5px 18px;
        margin: 8px 0;
        text-align: right;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        """
        prefix = "👤 You: "
        last_exchange['user_input'] = message
    else:
        style = """
        background: linear-gradient(135deg, #ff69b4 0%, #ff1493 100%);
        color: white;
        padding: 12px;
        border-radius: 18px 18px 18px 5px;
        margin: 8px 0;
        text-align: left;
        box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        """
        prefix = "🤖 Bot: "
        last_exchange['bot_response'] = message

    formatted_message = message.replace('\n', '<br>')

    chat_output.append_display_data(HTML(f"""
    <div style="{style}">
        <strong>{prefix}</strong><span style="color: white;">{formatted_message}</span>
    </div>
    """))

def on_thumbs_up(b):
    """Handle positive feedback"""
    if last_exchange['user_input'] and last_exchange['bot_response']:
        chatbot.collect_feedback(
            last_exchange['user_input'], 
            last_exchange['bot_response'], 
            'thumbs_up'
        )
        display_feedback_message(" Thanks for the positive feedback! I'll keep improving.")
    else:
        display_feedback_message(" Please chat with me first before giving feedback.")

def on_thumbs_down(b):
    """Handle negative feedback - show feedback input"""
    if last_exchange['user_input'] and last_exchange['bot_response']:
        display_feedback_message(" How can I improve? Type your feedback below and click 'Submit Feedback'")
        feedback_input.value = ""
    else:
        display_feedback_message(" Please chat with me first before giving feedback.")

def on_submit_feedback(b):
    """Submit detailed feedback"""
    if last_exchange['user_input'] and last_exchange['bot_response']:
        user_feedback = feedback_input.value.strip()
        if user_feedback:
            chatbot.collect_feedback(
                last_exchange['user_input'],
                last_exchange['bot_response'],
                'thumbs_down',
                user_feedback
            )
            display_feedback_message(" Thank you! Your feedback helps me learn and improve.")
            feedback_input.value = ""
        else:
            display_feedback_message(" Please provide some feedback text.")
    else:
        display_feedback_message(" Please chat with me first before giving feedback.")

def on_report_clicked(b):
    """Show performance report"""
    report = chatbot.get_performance_report()
    display_message("Bot", f" **Performance Analytics**\n\n{report}", False)

def display_feedback_message(message):
    """Display feedback-related messages"""
    style = """
    background: linear-gradient(135deg, #ffd700 0%, #ffa500 100%);
    color: black;
    padding: 8px;
    border-radius: 10px;
    margin: 5px 0;
    text-align: center;
    font-style: italic;
    """
    chat_output.append_display_data(HTML(f"""
    <div style="{style}">
        <strong> </strong>{message}
    </div>
    """))

def process_user_input(input_widget):
    user_text = input_widget.value.strip()
    if user_text:
        display_message("You", user_text, True)

        start_time = time.time()
        response = get_chatbot_response(user_text)
        end_time = time.time()
        response_time = (end_time - start_time) * 1000

        display_message("Bot", response, False)

        chat_output.append_display_data(HTML(f"""
        <div style="text-align: center; color: #888; font-size: 11px; margin: 2px 0; font-style: italic;">
             Response time: {response_time:.1f}ms
            &nbsp;|&nbsp;
             How was this response? 
            <span style="color: #4CAF50; cursor: pointer;" onclick="virtualClick('thumbs_up')">👍</span> 
            <span style="color: #F44336; cursor: pointer;" onclick="virtualClick('thumbs_down')">👎</span>
        </div>
        <hr style="border: none; border-top: 1px dashed #ddd; margin: 10px 0;">
        """))

        input_widget.value = ""

# Connect all events
send_btn.on_click(lambda b: process_user_input(user_input))
clear_btn.on_click(lambda b: [chat_output.clear_output(), display_message("Bot", "💫 Chat cleared! How can I help you find your next great read? 📚", False)])
user_input.on_submit(lambda x: process_user_input(x))

thumbs_up_btn.on_click(on_thumbs_up)
thumbs_down_btn.on_click(on_thumbs_down)
submit_feedback_btn.on_click(on_submit_feedback)
report_btn.on_click(on_report_clicked)

# Enhanced suggestion buttons
sample_authors = books_df['Author'].value_counts().head(2).index.tolist()
sample_titles = books_df['Title'].head(2).tolist()
sample_genres = books_df['Genre'].value_counts().head(2).index.tolist()

suggestions = []
if sample_authors:
    suggestions.append(f" Books by {sample_authors[0]}")
if sample_titles:
    suggestions.append(f" Price of {sample_titles[0]}")
if sample_genres:
    suggestions.append(f" {sample_genres[0]} books")

suggestions.extend([" Books under 500", " Books under 1000", " Books under 1500"])
suggestions.extend([" Opening hours", " Available genres", " Delivery options"])

suggestion_btns = []
for suggestion in suggestions:
    btn = widgets.Button(
        description=suggestion,
        layout=widgets.Layout(width='auto', margin='3px', height='35px'),
        button_style='info',
        style={'font_size': '12px'}
    )
    btn.on_click(lambda b, s=suggestion: setattr(user_input, 'value', s.replace("📚 ", "").replace("💰 ", "").replace("🎭 ", "").replace("💸 ", "").replace("💵 ", "").replace("🕒 ", "").replace("📖 ", "").replace("🚚 ", "")))
    suggestion_btns.append(btn)

# Display the complete GUI
display(HTML("""
<h3 style='
    text-align: center; 
    color: #ff69b4; 
    background: linear-gradient(135deg, #f0f8ff, #e6f3ff);
    padding: 15px;
    border-radius: 10px;
    margin: 10px 0;
    border: 2px solid #ff69b4;
'>📚 Book Haven Chatbot 
</h3>
"""))

display(chat_output)

# Input row with feedback buttons
input_row1 = widgets.HBox([user_input, send_btn, thumbs_up_btn, thumbs_down_btn, report_btn, clear_btn])
display(input_row1)

# Feedback input row
feedback_row = widgets.HBox([feedback_input, submit_feedback_btn])
display(feedback_row)

# Suggestions
display(HTML("<br><h4 style='color: #667eea;'> Quick Suggestions:</h4>"))
suggestions_box = widgets.HBox(suggestion_btns, layout=widgets.Layout(flex_wrap='wrap', margin='10px 0'))
display(suggestions_box)

# Welcome message explaining new features
display_message("Bot", """
🌟 Welcome to Book Haven Book Store! 🌟

I'm your intelligent assistant that **learns from your feedback**! 

 **Book Discovery Features:**
• Find books by author, title, or genre
• Check book prices and availability  
• Browse books by price range 
• Find affordable books within your budget

 **Store Services:**
• Opening hours & location
• Delivery options
• Payment methods

 **NEW: AI Feedback Learning System**
• Click 👍 if my response was helpful
• Click 👎 to help me improve
• I learn from your feedback and get smarter!
• View analytics with the  Report button

 **Try These Queries:**
• "Books under 1000"
• "Which African Literature books do you have?"
• "Do you have romance books?"
• "Books by your favorite author"

Let's find your next favorite book! 📚✨
""", False)



 LAUNCHING COMPLETE CHATBOT INTERFACE WITH FEEDBACK LEARNING


C:\Users\DELL\AppData\Local\Temp\ipykernel_10676\3773579658.py:195: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  user_input.on_submit(lambda x: process_user_input(x))


Output(layout=Layout(border_bottom='2px solid #ff69b4', border_left='2px solid #ff69b4', border_right='2px sol…